In [2]:
library(data.table)
library(stargazer)


Please cite as: 

 Hlavac, Marek (2015). stargazer: Well-Formatted Regression and Summary Statistics Tables.
 R package version 5.2. http://CRAN.R-project.org/package=stargazer 



# Regression to Estimate Causal Effects 

Recall Week 2 where we discussed potential outcomes. 

> The concept of potential outcomes are foundational to the reasoning system that we're going to use for the course. They allow us to think about, with considerably clarity, the comparisons that we want to make, and exactly why these comparisons are causal.

Here, we are going to remake that same data, and then estimate the *unknown* average treatment effect from two groups, using regression. 

## Create Data 

Let's make some toy data that we can use through this demo. 

Suppose that we have 1,000 individuals that we can observe in our study. Furthermore, suppose that for each of these people, there is some *latent*, or unmeasured, response to treatment that they *would* have to treatment if we assigned them to take the treatemnt. 

You might think of this as an unknowable population-level parameter that we're trying to estimate from empirical data. In this worksheet, we're going to make the data so that the causal parameter $\tau$ has a mean value of 2. But, you could imagine that some other treatment has a mean value of 100. Or, for some third treatment it could have a mean value of zero, in which case we would say there there is *no* treatment effect. 

For concreteness, suppose that the treatment is assigning people to drink [coffee](https://www.aldeacoffee.com). 



Create a data.table with the following columns: 

1. An `id` column that ranges from 1 to the number of rows in the data.table. 
2. A `tau` column that has random draws from the normal distribution. Set the mean of these draws to be two, and the standard deviation to also be two. 

In [3]:
NROWS <- 1000 

d <- data.table(

)

Let's build more into our data, like, that some of the people are tall, while others are short; and some are young, while others are old. 

Make two new colums, using the `:=` assignment operator in the data.table. We'll make the first column for you. You produce the second column. 

1. `Height` which is random draws of either 'tall' or 'short'. 
2. `Age` which is random draws of either 'young' or 'old'. 


In [4]:
d[ , height := sample(c('tall','short'), size = .N, replace = TRUE)]


In just exactly the same way, we can also think of our units having levels of *potential outcomes to control*. That is, we can suppose that people have some level of the outcome in the case that they do not drink any coffee. For concreteness, suppose that the outcome is people's number of minutes of 241 coding they can accomplish, before they fall asleep. 

There might (or might not) be a relationship between our non-experimentally assigned outcomes. In the example we're working with here, suppose that there is no relationship between height and minutes of coding; but that there is a positive relationship between age and minutes of coding. 

This might be represented in our data in the following way: The mean number of minutes that someone can work is 10, plus, if they're old (ahem... seasoned, disciplined) they are able to work for an additional 5 minutes on avergae. But, also assume that there is some noise in this relationship. 

Create a column on the data called `y_0` that takes the following values. 

- If the preson is young, it takes the value of 10. 
- If the person is old, it takes the value of 15. 
- For both young and old people add in a little bit of *noise* a a draw from a random normal distribution with `mean=0` and `sd=1`. 

There are a couple of ways that you can do this. While you may be tempted to subset your data; this is probably less efficient that it could be. 

- Either use an `ifelse` statement, which is still not maximally performant; or, 
- Use an evaluation statement using the `I()` "as is" operator. 

In [5]:
d[ , ]

Notice that there is no relationship between height and potential outcomes to control; and also notice that the "noise" in the relationship is represented in the draw from the normal distribution with mean 0. 

## Potential Outcomes to Treatment 
If we know people's potential outcomes to control, and we already know each persons *causal effect*, then I suppose we also know their potential outcomes to **treatment**, right? 

Create people's potential outcomes to treatment, `y_1` as the sum of their `y_0` and `tau` values. 

In [8]:
d[ , y_1 := ]

head(d)

id,tau,height,age,y_0,y_1
1,3.464029,tall,old,15.31163....,18.77566....
2,1.367973,tall,old,15.38792....,16.75589....
3,1.099995,tall,young,9.938720....,11.03871....
4,5.448695,tall,old,14.76511....,20.21381....
5,2.665182,tall,young,9.223416....,11.88859....
6,3.367683,tall,old,13.60667....,16.97435....


And so, we can represent this **science table** -- the set of all potentially realizable outcomes, and also measurable covariates. 

# Run Your Experiment 

To this point, we've been working in the *potential outcomes space*. One way that you might think about this is as though these are the measurement that our population of people are going to walk into the expeiment having -- but we haven't measured them yet, so we don't know what they are! That is, suppose we have the whole cohort of students who are enrolled in 241, and they're going to start the coffee drinking experiment in Week 3. In Week 2, they all are either young/old, short/tall, and have innate abilities to focus, *we as the experimenters just don't know them yet!*. 

When we run the experiment, we accomplish several things: 

1. We measure outcomes from our subjects;
2. We intervene in their lives to force a particular experience; 
3. As a result of our intervention, we *reveal* either potential outcomes to treatment or control for each subject, and we measure this. 

The first of these is easy: For every subject, no matter whether they are in treatment or control, we measure the trait that we care about. But, the second two require consierably more care, and are the focus of the class. 

In this experiment, the way that we are interviening in people's lives is to either give them *coffee* or *decaf coffee*. Then, and only then, we're also able to make some of their potential outcomes measurable. 


Make two final columns: 

1. Create variable `treat` and randomly assign people to be either in the treatment group or the control group. 
2. Create a variable `Y` that is the potential outcome that we *actually* observe in the experiment. If the person is assigned to the treatment group, fill `Y = y_1`; and, if they're assigned to the control group fill `Y = y_0`. 

## Why only some of? 

Why are we only able to measure some of people's potential outcomes? As David has identified in the async videos, we're only able to measure the potential outcomes that are consistent with the treatment that we actually give people. 

And, so, while the *science table* might contain information about each persons potential outcomes to treamtent and control, the table of data that we're ever going to be able to generate has a more restricted set. Call this observable data set `obs`, and it is a subset of all the data that might be out there.

In [7]:
obs <- d[ , .(id, height, age, treat, Y)]
head(obs)

ERROR: Error in eval(jsub, SDenv, parent.frame()): object 'treat' not found


Where did all the data go?? We no longer have access to people's causal effect ($\tau$), nor do we have access to their potential outcomes to control and treatment. Instead, we're left only with their *realized potential outcomes* that match the condition they were assigned to. 

# Estimating Causal Quantities from the Observable Data 

Of course, we don't get to see each of these potential outcomes
instead, we only get to see either one or the other. Which one do we
get to see? Well the potential outcome for the treatment condition that
we assign the person to! 

But, though random assignment, the values for the potential outcomes to control that we see *should* be an unbiased estiamte of the whole populations potential outcomes to control.

In the past, we have used differences in means, and randomization inference to make these estimates. However, in this week's work, we have said that a regression will accomplish the *exact same estimate*. 

## First, the difference in means estimate 

So that we have a baseline -- and so that we can solidify our understanding from previous week -- quickly estimate the treatment effect using a difference in means estimator. 

- Create a new variable `group_mean` that is the mean of the outcome...
- ...grouped by the treatment indicator.
- Then calculate the difference between the values in `group_mean`. 

## Now, the regression estimate 

With that baseline built, now estimate the same quantity, but using a regression. 

> For those of you coming back to R recently, the **formula** interface is integrated throughout the `R` ecosystem. On the left-hand side of a tilde (~) is the response variable, and on the right-hand side are the model variables. 

In [ ]:
# formula documentation is below if you're interested
# ?formula

If you'd like a longer-treatment of this, there's a nice [blog post that is maintained at the R-Studio community](https://rviews.rstudio.com/2017/02/01/the-r-formula-method-the-good-parts/). 

Using a regression, drawn from the function `lm` (for linear model), estimate the average difference between the treatment and control groups. 

In [ ]:
mod <- obs[ , lm(Y ~ treat)]
summary(mod)

The summary method of a fit `lm` object reports the coefficients, standard errors, and t-statistics, and p-values associated with those t-statistics. 

## Questions of Understanding 
1. In the model that you've just estimated, what information is estimated in the `(Intercept)` coefficient? 
2. In the model that you've just estimated, what information is estimated in the `treat` coefficient? What about this experiment **ensures** that this estimate is a causal estimate? Is it: 
  1. That the regression has conditioned away the other confounding features? Or, 
  2. That the design of the experiment has built randomiation so that there cannot possibly be other confounding features? 

## Improving Efficiency (Sort of...) 

After we are able to design an experiment that will *reliably* identify a causal effect, our next consideration is how we can generate as much power as possible from the experiment. One way that we have talked about in the lectures is to include other *non-causal* variables that might predict values of the outcome variable. 

Estimate a new model, that includes `height` as a *right-hand side predictor* (or feature) in this model. 

## Question of Understanding 
- What happens to your estimate of the treatment effect? 
  - Does it change any? 
  - Does it change *much*? 
- Has the precision of your estimate changed any? Use the `confint()` method on the old model that you estimated and the new model that you just estimated. Have the confidence intervals changed appreciably between the two? Why or why not? 
- Consider the coefficient on `height`. Is there any relationship between height and performance on the outcome attention? (Since you built the data, you know the answer to this without even estimating the relationship.) Is this borne out in the estimate? 


## Improving Efficiency (For real this time)

Although height doesn't seem to do much for our model, we have other features in the data that **are** predictive out outcomes -- namely, `age`. Estimate a model just like the one that you previously estimated with `height`, but this time using the `age` variable. Store this in a model, and then print the summary method of that model. 

## Question of Understanding 
- What happens to your estimate of the treatment effect? 
  - Does it change any? 
  - Does it change *much*? 
- Has the precision of your estimate changed any? Use the `confint()` method on the old model that you estimated and the new model that you just estimated. Have the confidence intervals changed appreciably between the two? Why or why not? 
- Consider the coefficient on `age`. Is there any relationship between height and performance on the outcome attention? (Since you built the data, you know the answer to this without even estimating the relationship.) Is this borne out in the estimate? 

